In [1]:
import sys
import pandas as pd
import numpy as np
import scanpy as sc
import pathlib as pt
import matplotlib as mpl
import matplotlib.pyplot as plt

scr_path = '/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/snippets'
sys.path.append(scr_path)
import singleCellFun as sf

f,_ = sf.makeOutoutFolders()
sf.nbInit(f)

# output path

In [2]:
output_path = pt.Path().absolute().parent / 'cpdh_input_T21_vs_2n'
print(output_path)

/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/matching_DS_Oct24/all_soupX/T21_vs_2n/cpdh_T21_vs_2n/cpdh_input_T21_vs_2n


# read celltype edgeR DGE

read up regulated genes,
concatenate lists, 
make celltype - gene list

In [ ]:
# THRESH 
FDR_THRESH = 0.05
LOGFC_THRESH = 0.2
MIN_GENES_THRESH = 20
MIN_PER_THRESH = 10

FDRstr = str(FDR_THRESH).replace('.','')
LOGFCstr = str(LOGFC_THRESH).replace('.','')
MIN_GENESstr = str(MIN_GENES_THRESH)
MIN_PERstr = str(MIN_PER_THRESH)

In [22]:
# load DEG
fname = '../../data/combined_DEG_results_forHassan.csv'
df = pd.read_csv(fname)
print(f'{df.shape} significant genes')

dge = df[(df['logFC'] > LOGFC_THRESH) & (df['FDR'] < FDR_THRESH)]

# genes expressed in late age-group in both cell types
dge = dge[dge['comparison'].isin(['fTFC1_14_20','fTFC2_14_20','analysis2_fTFC1','analysis2_fTFC2'])]
# dge = dge[dge['comparison'].isin(['fTFC1_11_13'])]
dge['cell_type'] = dge['comparison']

update = {'analysis2_fTFC1':'thy_TH_processing', 'analysis2_fTFC2' : 'thy_Lumen_forming',
          'fTFC1_14_20':'thy_TH_processing', 'fTFC2_14_20' : 'thy_Lumen_forming'}
dge = dge.replace(update)
# reorder columns
dge = dge.rename(columns={'geneSym':'gene'})
col = dge.columns
cols2front = ['cell_type','gene']
dge = dge[cols2front + list(set(col)-set(cols2front))]
# concat celltypes - karyotypes
dge['cell_type'] = dge['cell_type'] + str('.') + str('T21')
# print
print(dge.shape)
print(dge['cell_type'].unique())
display(dge.head(3))

(365, 17) significant genes
(294, 18)
['thy_TH_processing.T21' 'thy_Lumen_forming.T21']


,cell_type,gene,chr,isCosmic,cosmicTier,comparison,F,isCSM,FDR,logFC,isTSG,PValue,logCPM,ensID,isTF,Unnamed: 0,tumourType,direction
0,thy_TH_processing.T21,PDXK,chr21,False,NaN,thy_TH_processing,143.840007,False,0.000063,0.904324,False,9.560567e-09,5.666743,ENSG00000160209,False,ENSG00000160209,NaN,T21_up
1,thy_TH_processing.T21,AGPAT3,chr21,False,NaN,thy_TH_processing,142.627732,False,0.000063,0.955210,False,1.009336e-08,6.036753,ENSG00000160216,False,ENSG00000160216,NaN,T21_up
2,thy_TH_processing.T21,USP16,chr21,False,NaN,thy_TH_processing,115.505041,False,0.000161,0.635044,False,3.848646e-08,6.881284,ENSG00000156256,False,ENSG00000156256,NaN,T21_up


In [5]:
fname = f'dge_mergeMi_T21_vs_2n_{MIN_GENESstr}min_{MIN_PER_THRESH}per_p1Luz.tsv'
print(f'output filename: {fname}')
dge.to_csv(output_path / fname, index=False, sep='\t')

output filename: dge_mergeMi_T21_vs_2n_20min_10per_p1Luz.tsv


# read microenvironment
read adata, subset for interesting celltypes, export adata, export metadata, export microenv

In [7]:
# run cellphone for all celltypes except of cycling cells
adata = sc.read('../../../workingObj/cdata_all_T21vs2n.h5ad')
sf.summaryTable(adata,['karyotype','pcw','donor','sex','cells'])

[INFO] - total cells: 131497


/nfs/team292/hm11/endo_GLND/starsolo/GLND/Thyroid/snippets/singleCellFun.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try: tempAnn['pcw'] = tempAnn['pcw'].values.astype(int)


,sex,pcw,karyotype,donor,cells
0,male,11,2n,Hrv188,9202
1,male,11,2n,Hrv195,5052
2,male,11,T21,Hrv256,3347
3,male,11,T21,Hrv257,4814
4,female,12,2n,Hrv197,5462
5,female,12,T21,Hrv265,5481
6,male,12,2n,Hrv116,3358
7,male,12,2n,Hrv218,5612
8,male,12,T21,Hrv266,5009
9,female,14,2n,Hrv194,7355


In [8]:
# zoom in to thy|eno|mes
cluster_in = ['thy_TH_processing','thy_Lumen_forming',
              'end_Lymphatic','end_Capillary','end_Venous','end_Arterial',
              'mes_CYGB']
cluster_in = [s for s in cluster_in if 'cycling' not in s.lower()]
cluster_in

['thy_TH_processing',
 'thy_Lumen_forming',
 'end_Lymphatic',
 'end_Capillary',
 'end_Venous',
 'end_Arterial',
 'mes_CYGB']

In [9]:
# update fields (replace '-' by '_')
adata.obs['celltype'] = adata.obs['celltype'].replace({'thy_Lumen-forming' : 'thy_Lumen_forming', 'epi_HLA-B' : 'epi_HLA_B'})
# subset adata
adata = adata[adata.obs['celltype'].isin(cluster_in)]
adata.shape

(108750, 30997)

# write metadata

In [10]:
tmp = adata.obs['celltype'].astype(str) + '.' + adata.obs['karyotype'].astype(str)
tmp.reset_index().to_csv(output_path / 'cdata_all_MATCHING_metadata_p1Luz.tsv', index=False, sep='\t')

# write microenv

In [11]:
pd.concat([tmp, adata.obs['karyotype']], axis=1).drop_duplicates().reset_index(drop=True).to_csv(output_path / 'cdata_all_MATCHING_microenvironment_p1Luz.tsv',index=False, sep='\t')